In [49]:
from sklearn.linear_model import SGDClassifier
import numpy as np
import collections
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch import Tensor
import geometry
import itertools
from collections import defaultdict
import collections
import target
import controler

def cumsum(vector):
    cumulative = []
    sum =0
    for i in vector:
        if i == None:
            cumulative.append(None)
        else:
            sum += i
            cumulative.append(sum)
    return cumulative



def STAP(clf, controler_input, nb_mistakes, i):    

    prediction = clf(controler_input)
    initial_action = torch.round( nn.Sigmoid()(prediction) )

    r = np.random.uniform(0,1)
    flip_proba = np.sqrt( (1+nb_mistakes)/(i+1) )

    if (initial_action == 1 ) or (initial_action == 0 and r <= flip_proba):
        STAP_action = 1
    else:
        STAP_action = 0
    
    return initial_action, STAP_action

def CesaBianchi(clf, controler_input, beta, K):

    prediction = clf(controler_input)
    initial_action = torch.round( nn.Sigmoid()(prediction) )

    p = beta / ( beta + abs( prediction.item() ) )
    cesa_action = np.random.binomial(1, p)
    
    
    return initial_action, cesa_action
    

def PLOT(epsilon,):
    q = np.random.binomial(1, epsilon)


def generate_sequence(target, k, t, test_loader,n_classes,epsilon):

        R = {}
        Sa = []
        for i,data in enumerate(test_loader):
            x,y = data
            x_attacked = fgsm_attack(target, epsilon, x, y, n_classes)

            if i < t:
                v =  nn.CrossEntropyLoss()( target.clf(x_attacked) ,y)
                R[i] = v
                R = {k: v for k, v in sorted(R.items(), key=lambda item: item[1])}
            else:
                v =  nn.CrossEntropyLoss()( target.clf(x_attacked) ,y)
                last_value = list(R.values())[-1]
                last_key = list(R.keys())[-1]
                if v>=last_value and len(Sa)<=k:
                    del R[ last_key ]
                    R[i] = v
                    R = {k: v for k, v in sorted(R.items(), key=lambda item: item[1])}
                    Sa.append(i)

        attack_label = [ 1 if i in Sa else 0 for i in range(len(test_loader)) ]
                    
        return attack_label

def fgsm_attack(target, epsilon, x, y, n_classes):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
    
    if n_classes <=2:
        dlt =  target.clf(x)[0] - y
    else:
        label = torch.zeros(n_classes)
        label[y] = 1
        dlt =  target.clf( x ) - label

    direction = torch.sign( torch.matmul( dlt, target.clf[0].weight ) ).to(device)
    x_attacked = x + epsilon * direction
    return x_attacked

def reset(clf):
    for layer in clf.children():
        if hasattr(layer, 'reset_parameters'):
            layer.reset_parameters()




In [5]:
class Task:

    def __init__(self, n, n_features, n_classes, class_sep, percent_flip, attack_proba, epsilon, seed = 42):
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
        self.attack_proba = attack_proba

        self.n_features, self.n_classes = n_features, n_classes
        X, y = make_classification(n_samples = n, n_features = n_features,  n_informative = 2, n_redundant = 1, n_classes = n_classes, flip_y = percent_flip, class_sep = class_sep, n_clusters_per_class = 1 )
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=seed)
        self.X_train, self.y_train = X_train, y_train
        train_loader = DataLoader( TensorDataset( Tensor(X_train).to(self.device), Tensor(y_train).type(torch.LongTensor).to(self.device) ) , batch_size = 32, shuffle=True)
        test_loader = DataLoader(  TensorDataset( Tensor(X_test).to(self.device), Tensor(y_test).type(torch.LongTensor).to(self.device) ), batch_size = 1, shuffle=False)

        ### Train the target classifier:
        self.target = target.Target(n_classes,n_features)
        self.target.train(train_loader, 10)

        #attack_label = generate_sequence(self.target, 10, 10, test_loader, n_classes, epsilon)
        attack_label = [  np.random.choice([1,0], p=[attack_proba, 1-attack_proba]) for _ in range(len(X_test)) ]
        X_test = [ fgsm_attack(self.target, epsilon, data[0], data[1], n_classes) if attack_label[i] else data[0] for i,data in enumerate(test_loader) ]

        self.test_loader = DataLoader(  TensorDataset( torch.stack( X_test ).to(self.device), Tensor(y_test).type(torch.LongTensor).to(self.device) ), batch_size = 1, shuffle=False)
        self.attack_label = torch.Tensor(attack_label).to(torch.float32)

                    
    # def offline(self):

    #     attack_clf = Controler( 2, self.n_features)
    #     #train_attack_label = generate_sequence(self.target, 10, 10, test_loader, n_classes, epsilon)
    #     train_attack_label = [  np.random.choice([1,0], p=[self.attack_proba, 1-self.attack_proba]) for _ in range(len(self.X_train)) ]
    #     train_loader = DataLoader( TensorDataset( Tensor(self.X_train).to(self.device), Tensor(self.train_attack_label).type(torch.LongTensor).to(self.device) ) , batch_size = 1, shuffle=True)
    #     X_train_attacks = [ fgsm_attack(self.target, epsilon, data[0], data[1], n_classes) if train_attack_label[i] else data[0] for i,data in enumerate(train_loader) ]
    #     train_loader = DataLoader( TensorDataset( Tensor(X_train_attacks).to(self.device), Tensor(self.y_train).type(torch.LongTensor).to(self.device) ) , batch_size = 1, shuffle=True)
        
    #     attack_clf.train(  train_loader, n_epochs=30)  
    #     y_pred = attack_clf.predict( self.X_test )

    #     tn, fp, fn, tp = confusion_matrix( self.attack_label , y_pred).ravel()
    #     print('True positive:{}, False positive:{}, Attacks:{}'.format(tp,fp,sum(self.attack_label) ) )

    def online(self, method, beta=1 ):

        control = controler.Controler( 2, self.n_features+self.n_classes)
        regrets = []
        K, attacks,i, abs_false_pos, abs_true_pos, false_pos, true_pos,  = 0,0,0,0,0,0,0
        for i,data in enumerate(self.test_loader):

            x, y_true = data[0][0], data[1] 
            is_attacked = self.attack_label[i]

            target_pred = self.target.clf(x) 
            controler_input = torch.concat( (x, target_pred), axis = 1)

            if method == 'STAP':
                initial_action, decision = STAP(control.clf, controler_input, K, i) 
            elif method == 'Cesa':
                initial_action, decision = CesaBianchi(control.clf, controler_input, beta, K)

            if decision == 1:
                if is_attacked != initial_action:
                    control.collect_fit([controler_input.detach()], is_attacked)
                    K = K+1  

            abs_true_pos = abs_true_pos+1 if decision == 1 and is_attacked == 1 else abs_true_pos
            abs_false_pos = abs_false_pos+1 if decision == 1 and is_attacked == 0 else abs_false_pos
            true_pos = true_pos+1 if initial_action == 1 and is_attacked == 1 else true_pos
            false_pos = false_pos+1 if initial_action == 1 and is_attacked == 0 else false_pos
            regret = abs( decision - is_attacked)

            regrets.append(regret.item())
            attacks = attacks+1 if is_attacked == 1 else attacks
            i+=1

        plt.plot( cumsum(regrets) )
        plt.xlabel('Iteration')
        plt.ylabel('Cumulative Regret')
        print('     True positive:{},      False positive:{}'.format(true_pos,false_pos,attacks))
        print('Abs. True positive:{}, Abs. False positive:{}, Attacks:{}'.format(abs_true_pos,abs_false_pos,attacks))

    def f(self, alpha, t):
        return (alpha ** 1/3) * (t**2/3) * ( np.log(t)**1/3 )

    def W_k(L, H, pair,mathcal_K_plus):
        v_ij = geometry.observer_vector(L, H, pair[0], pair[1], mathcal_K_plus)
        return max( [ np.linalg.norm( v_ij[k],'inf') for k in mathcal_K_plus ]  )

    def cpb_side(self,):

        controlers = [ controler.Controler( 2, self.n_features+self.n_classes), controler.Controler( 2, self.n_features+self.n_classes) ]
        # alpha = 1.1 #alpha > 1
        L = [ [0,1],[1,0] ]
        H = [ [None,None],[0,1] ]
        mathcal_N = [0,1]
        mathcal_M = [0,1,None]
        N = len(L)
        n = np.zeros(N)
        # mathcal_P = [ a for a in mathcal_N if geometry.isParetoOptimal(1, L)] # set of pareto optimal actions
        # mathcal_K = [ pair for pair in list( itertools.combinations([0,1], 2) ) if geometry.areNeighbours(pair[0], pair[1], L) ] #set of unordered neighboring actions
        
        V_ij, v_ijk, W_k, r = [0]*4

        for t,data in enumerate(self.test_loader):
            x, y_true = data[0][0], data[1] 
            target_pred = self.target.clf(x) 
            controler_input = torch.concat( (x, target_pred), axis = 1)
            half_space = collections.defaultdict(dict)

            if t < mathcal_N:  # initialisation
                I  = t
                n[t] += 1
                J = H[ I ][ y_true ]
                #print(str(I)+' '+str(J) )
                controlers[t].collect_fit([controler_input.detach()], J)

            else:
                q,w = np.zeros(N), np.zeros(N)
                for i in mathcal_N:
                    q[i] = controlers[i].clf(controler_input)  
                    w[i] =  controlers[i].confidence()

                for pair in mathcal_K:
 
                    mathcal_K_plus =  geometry.Neighbourhood(pair[0], pair[1], L) #neighborhood action set of pair 
                    v_ij = geometry.observer_vector(L, H, pair[0], pair[1], mathcal_K_plus)
                    d_ij = sum( [  v_ij[k].T @ q[k]   for k in mathcal_K_plus ] )
                    c_ij = sum( [  np.linalg.norm( v_ij[k] ) @ w[k]   for k in mathcal_K_plus ] )
                    if abs( d_ij ) >= c_ij:
                        half_space[ pair[0] ][ pair[1] ] = np.sign(d_ij)
                    else:
                        half_space[ pair[0] ][ pair[1] ] = 0

                    mathcal_P_t, mathcal_K_t = geometry.get_polytope(half_space, L, mathcal_N, mathcal_P, mathcal_K)

            mu_k = 0.1
            K_plus_t = np.concatenate( [ geometry.Neighbourhood(pair[0], pair[1], L) for pair in mathcal_K_t ] )
            mathcal_V_t = np.concatenate( [  geometry.observer_vector(L, H, pair[0], pair[1], mathcal_K_plus) for pair in mathcal_K_t ] )
            mathcal_R_t = np.array( [ k for k in mathcal_N if n[k]<= mu_k * self.f(alpha, t) ] )
            mathcal_S_t = np.concatenate( [ mathcal_P_t , K_plus_t , np.intersect1d(mathcal_V_t, mathcal_R_t) ] )
            values = [ W_k(L, H, pair,mathcal_K_plus)/n[i] for i in mathcal_S_t ]
            I = np.argmax(values)
            if I in np.setdiff1d( mathcal_V_t, np.concatenate([mathcal_P_t,K_plus_t]) ):
                r += 1
            J = H[ I ][ y_true ]
            controlers[ I ].collect_fit([controler_input.detach()], J)        
    

SyntaxError: invalid syntax (3305853621.py, line 206)

In [ ]:


n = 1000
n_classes = 2
classes =  np.arange(0, n_classes)
attack_proba =  0.1
epsilon = 10
n_features = 10
class_sep = 3
percent_flip = 0.1


task = Task(n, n_features, n_classes, class_sep, percent_flip, attack_proba, epsilon)
task.cpb_side()

# task.online('STAP')
# task.online('Cesa', 3)
# print()
#print("Offline:")
#task.offline()

In [7]:
class PMGame:

    def __init__(self, L, H):

        self.LossMatrix = L
        self.FeedbackMatrix = H
        self.N = L.shape[0] # Number of learner actions
        self.M = H.shape[0] # Number of environment outcomes
        self.Actions_dict = { a : "{0}".format(a) for a in range(self.N)} # Actions semantic
        self.Outcomes_dict = { a : "{0}".format(a) for a in range(self.M)} # Outcomes semantic





L = np.array([ [0,1],[1,0] ])
H = np.array([ ['no_feedback','no_feedback'],[0,1] ])

# pm = PMGame(L,H)
# geometry.ProblemClass(pm)

# print("Loss matrix:")
# print(L)
# for i in range(L.shape[0]):
#     print()
#     print("Domination matrix for action", i, ":" )
#     print(geometry.domination_matrix(i,L) )
#     print("Domination polytope:", geometry.DominationPolytope(i,L).minimized_generators() )
#     print("dominating:", geometry.isNonDominated(i,L) )
#     print("Strict Domination polytope:", geometry.StrictDominationPolytope(i,L).minimized_generators() )
#     print("strictly dominating:", geometry.isStrictlyNonDominated(i,L) )
#     print("degenerated:", geometry.isDegenerated(i,L) )
#     print("Pareto Optimal:", geometry.isParetoOptimal(i, L) )


# geometry.global_signal(H)
# mathcal_K = [ pair for pair in list( itertools.permutations([0,1], 2) ) if geometry.areNeighbours(pair[0], pair[1], L) ] #set of unordered neighboring actions
# mathcal_K